# Code Execution with Gemini

[Lab link](https://partner.cloudskillsboost.google/paths/2294/course_templates/1235/labs/527178)

Note: this ipy-notebook was created step-by-spet according to the lab instructions (there is no a template file is ready to use)

### Objective

In this lab, you will learn how to generate and execute code using the Gemini API in Vertex AI and the Google Gen AI SDK for Python with the Gemini 2.0 Flash model.

You will complete the following tasks:
 - Generate and run sample Python code from text prompts.
 - Explore data using code execution in multi-turn chats.
 - Use code execution in streaming sessions.

 ### Task 1. Initialize the Google Gen AI SDK in a Colab Enterprise notebook

In [ ]:
# Install the Google Gen AI SDK and restart the kernel
%pip install --upgrade --quiet google-

# After the cell completes running, indicated by a checkmark to the left of the cell, 
# the new packages should be installed. To use them, restart the runtime by selecting Runtime 
# from the menu bar and clicking Restart Session. 
#
# When asked to confirm, select Yes. The runtime will restart, indicated by clearing the green checkmark 
# and the integer next to the cell above representing the order in which the cells have run.

### Task 2. Initialize the Google Gen AI SDK

In [ ]:
import os

from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import GenerateContentConfig, Tool, ToolCodeExecution, Image

In [ ]:
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Task 3. Generate and execute code with Gemini 2.0

In [ ]:
# initialize the code execution tool you imported above:
code_execution_tool = Tool(code_execution=ToolCodeExecution())

In [ ]:
# Define a prompt to instruct the model to generate code. Note that the model decides 
# when to use tools such as code generation (as opposed to returning code as text, 
# for example), so your prompt is important to direct the model to do what you would like:

PROMPT = """
What is the sum of the first 50 prime numbers?
Generate and run code for the calculation.
"""

In [ ]:
# Run the following to send the prompt to the model. 
# Note that the code execution tool is passed to the request so the model can use it:

response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=PROMPT,
    config=GenerateContentConfig(
        tools=[code_execution_tool],
        temperature=0,
    ),
)    

In [ ]:
# The following code iterates through the response and displays any generated Python code
# by checking for part.executable_code in the response parts:

for part in response.candidates[0].content.parts:
    if part.executable_code:
        print(part.executable_code.code)


'''
Output:

def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

primes = []
num = 2
while len(primes) < 50:
    if is_prime(num):
        primes.append(num)
    num += 1

sum_of_primes = sum(primes)
print(f'{sum_of_primes=}')
'''

In [ ]:
# View the code execution results.
# The following code iterates through the response and displays the execution result 
# and outcome by checking for part.code_execution_result in the response parts:

for part in response.candidates[0].content.parts:
    if part.code_execution_result:
        display(Markdown(f"`{part.code_execution_result.output}`"))
        print("\nOutcome:", part.code_execution_result.outcome)    

'''
Output:

sum_of_primes=5117

Outcome: Outcome.OUTCOME_OK
'''

### Task 4. Write a function to handle various parts in the response

In [ ]:
# You don't want to have to repeat code blocks like those cells above that iterate through the function's response.
# This is especially true if you consider that those blocks would be even longer if you also displayed 
# the text or inline data that the model can return.
#
# Instead, it can often be a good idea to define a function to handle the model's responses. 
# The following function will help you display:
#  - text sent as part.text
#  - generated images sent as inline data by checking for part.inline_data and then part.inline_data.mime_type to understand the file type and part.inline_data.data to receive the data itself
#  - generated Python code and execution results by checking for part.executable_code and part.code_execution_result:
    
# Run the following code in a new cell to define the handle_response function:

def handle_response(response):
    for part in response.candidates[0].content.parts:
        # For text responses
        if part.text:
            display(Markdown(part.text))

        # For generated inline image responses
        if part.inline_data:
            if part.inline_data.mime_type.startswith("image"):
                Image(image_bytes = part.inline_data.data).show()            

        # For executable code responses (code)
        if part.executable_code:
            print(part.executable_code.code)
        
        # For executable code responses (results)
        if part.code_execution_result:
            display(Markdown(f"`{part.code_execution_result.output}`"))
            print("\nOutcome:\n")
            display(Markdown(part.code_execution_result.outcome))
